Importing all the required libraries

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


Reading the CSV File aand breaking the DateTime to seasons, weekday

In [ ]:


df = pd.read_csv("/content/sample_data/Part1_Crime_data.csv")
df.shape

df['CrimeDateTime'] = df['CrimeDateTime'].astype('datetime64[ns]')
df['Crime_Year']=df.CrimeDateTime.dt.year
df['Crime_Month']=df.CrimeDateTime.dt.month_name()
df['Crime_Day']=df.CrimeDateTime.dt.day
df['Crime_Hour']=df.CrimeDateTime.dt.hour
df['Crime_WeekDay'] = df.CrimeDateTime.dt.weekday
print(df['Crime_WeekDay'])


0         5
1         5
2         5
3         5
4         5
         ..
434539    2
434540    2
434541    2
434542    2
434543    2
Name: Crime_WeekDay, Length: 434544, dtype: int64


Categorizing the Crime types into Low Medium and High (0,1,2)

In [ ]:

replace_val = "0"
df.replace("LARCENY", replace_val, inplace=True)
df.replace("LARCENY FROM AUTO", replace_val, inplace=True)
df.replace("AUTO THEFT", replace_val, inplace=True)
df.replace("BURGLARY", replace_val, inplace=True)

replace_val = "1"
df.replace("ROBBERY", replace_val, inplace=True)
df.replace("ROBBERY - STREET", replace_val, inplace=True)
df.replace("ROBBERY - COMMERCIAL", replace_val, inplace=True)
df.replace("ROBBERY - RESIDENCE", replace_val, inplace=True)
df.replace("ROBBERY - CARJACKING", replace_val, inplace=True)
df.replace("ASSAULT", replace_val, inplace=True)
df.replace("COMMON ASSAULT", replace_val, inplace=True)
df.replace("AGG. ASSAULT", replace_val, inplace=True)

#Fatal
replace_val = "2"
df.replace("SHOOTING", replace_val, inplace=True)
df.replace("RAPE", replace_val, inplace=True)
df.replace("ARSON", replace_val, inplace=True)
df.replace("HOMICIDE", replace_val, inplace=True)




Categorizing months into seasons

In [ ]:



#changing the labels of month to quantitative values
#1-Spring
#2-Summer
#3-Rainy
#4-Winter
df.replace("January", 4, inplace=True)
df.replace("February", 4, inplace=True)
df.replace("March", 1, inplace=True)
df.replace("April", 1, inplace=True)
df.replace("May", 1, inplace=True)
df.replace("June", 2, inplace=True)
df.replace("July", 2, inplace=True)
df.replace("August", 2, inplace=True)
df.replace("September", 3, inplace=True)
df.replace("October", 3, inplace=True)
df.replace("November", 3, inplace=True)
df.replace("December", 4, inplace=True)

df['Crime_WeekDay'] = np.where((df['Crime_WeekDay']) < 5,0,1)

df = df.rename(columns={'Description': 'Level', 'Crime_Month': 'Season'})


col_list = ['Season','Crime_Hour','District','Neighborhood','Crime_WeekDay']

df2 = df[col_list]

#factorize independent variables:



Factorizing independent variables

In [ ]:

#factorize weekday
weekday_var = pd.factorize(df2['Crime_WeekDay'])
df2['Crime_WeekDay'] = weekday_var[0]
weekday = weekday_var[1] 

#factorize Season:
season_var = pd.factorize(df2['Season'])
df2['Season'] = season_var[0]
season = season_var[1] 

#factorize Crime_Hour:
hour_var = pd.factorize(df2['Crime_Hour'])
df2['Crime_Hour'] = hour_var[0]
definition_list_Crime_Hour = hour_var[1] 

#factorize District:
district_var = pd.factorize(df2['District'])
df2['District'] = district_var[0]
district = district_var[1] 

#factorize Neighborhood:
neighbourhood_var = pd.factorize(df2['Neighborhood'])
df2['Neighborhood'] = neighbourhood_var[0]
neighborhood = neighbourhood_var[1]


y=df['Level']
X=df2


/var/folders/lw/8n3p52954nqb3s4smlypqtvr0000gn/T/ipykernel_88151/1544842150.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Crime_WeekDay'] = premise_var[0]
/var/folders/lw/8n3p52954nqb3s4smlypqtvr0000gn/T/ipykernel_88151/1544842150.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Season'] = month_var[0]
/var/folders/lw/8n3p52954nqb3s4smlypqtvr0000gn/T/ipykernel_88151/1544842150.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [ ]:
df2.head

<bound method NDFrame.head of         Season  Crime_Hour  District  Neighborhood  Crime_WeekDay
0            0           0         0             0              0
1            0           0         0             0              0
2            0           0         0             0              0
3            0           1         1             1              0
4            0           2         2             2              0
...        ...         ...       ...           ...            ...
434539       3           7         7           237              1
434540       3          21         6           165              1
434541       3           4         4            41              1
434542       3           4         6           106              1
434543       3          17         7            12              1

[434544 rows x 5 columns]>

Splitting the Train and Test Data


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

Created the RandomForestClassifier using GridSearchCV

In [ ]:
random = RandomForestClassifier()
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 31))
param_grid = dict(n_estimators=k_range)
# param_grid = [
#   {'max_depth': [2, 5, 10],
#    'min_samples_split':[3, 5, 10],
#    'class_weight':[None, 'balanced'],
#    'n_estimators':[10,20]
#   }
#  ]
  
# defining parameter range
grid = GridSearchCV(random, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(x_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


In [ ]:
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

{'n_estimators': 30}
Accuracy for our training dataset with tuning is : 58.17%


In [ ]:
print(classification_report(y_test, grid_search.predict(x_test)))

              precision    recall  f1-score   support

           0       0.63      0.74      0.68     50248
           1       0.49      0.40      0.44     34054
           2       0.12      0.02      0.03      2607

    accuracy                           0.58     86909
   macro avg       0.41      0.39      0.38     86909
weighted avg       0.56      0.58      0.57     86909



Creating the KNN Classifier with the same Data for train and Test

In [ ]:
knn = KNeighborsClassifier()

k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(x_train, y_train)

print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

#Printing the Classification Report
print(classification_report(y_test, grid_search.predict(x_test)))